<a href="https://colab.research.google.com/github/Murphylee140808/AI-Future-Directions/blob/main/AI_Future_wk6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
import pathlib
import os

# Set dataset paths
train_dir = "/content/dataset/dataset/train/resized_images"
val_dir = "/content/dataset/dataset/validation/resized_images"
test_dir = "/content/dataset/dataset/test/resized_images"

# Automatically detect class names from subfolders
train_data_dir = pathlib.Path(train_dir)
class_names = [item.name for item in train_data_dir.glob("*") if item.is_dir()]
print("Detected classes:", class_names)

# Load datasets
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

def load_dataset(directory):
    return tf.keras.preprocessing.image_dataset_from_directory(
        directory,
        labels="inferred",
        label_mode="int",
        image_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        shuffle=True
    )

train_ds = load_dataset(train_dir)
val_ds = load_dataset(val_dir)
test_ds = load_dataset(test_dir)

# Normalize images
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))
test_ds = test_ds.map(lambda x, y: (normalization_layer(x), y))

# Build a lightweight model for Edge AI
model = models.Sequential([
    layers.Conv2D(16, (3,3), activation='relu', input_shape=(224,224,3)),
    layers.MaxPooling2D(),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(len(class_names), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
EPOCHS = 5
history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Evaluate on test set
test_loss, test_acc = model.evaluate(test_ds)
print("Test Accuracy:", test_acc)

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = "recyclable_model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"TFLite model saved to {tflite_model_path}")

# Example inference on TFLite
interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Use the first image from test set
for images, labels in test_ds.take(1):
    test_image = images[0:1].numpy()
    interpreter.set_tensor(input_details[0]['index'], test_image)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    predicted_class = class_names[tf.argmax(output_data[0])]
    print("Predicted class:", predicted_class, "Actual class:", class_names[labels[0].numpy()])

Organizing dataset in: /content/dataset/dataset/train/resized_images
No images found in /content/dataset/dataset/train/resized_images. Skipping organization.
Organizing dataset in: /content/dataset/dataset/validation/resized_images
No images found in /content/dataset/dataset/validation/resized_images. Skipping organization.
Organizing dataset in: /content/dataset/dataset/test/resized_images
No images found in /content/dataset/dataset/test/resized_images. Skipping organization.
Detected classes: ['class_1', 'class_10', 'class_11', 'class_12', 'class_2', 'class_3', 'class_4', 'class_7', 'class_8', 'class_9']
Found 1000 files belonging to 10 classes.
Found 200 files belonging to 10 classes.
Found 200 files belonging to 10 classes.
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 52s 1s/step - accuracy: 0.2126 - loss: 2.2839 - val_accuracy: 0.3600 - val_loss: 1.6515
Epoch 2/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 44s 1s/step - accuracy: 0.4084 - loss: 1.5730 - val_accuracy: 0.4850 - val_loss: 1.5398
Epoch 3/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 0.5081 - loss: 1.4018 - val_accuracy: 0.4800 - val_loss: 1.5502
Epoch 4/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.5821 - loss: 1.2128 - val_accuracy: 0.4500 - val_loss: 1.5664
Epoch 5/5
32/32 ━━━━━━━━━━━━━━━━━━━━ 82s 1s/step - accuracy: 0.6057 - loss: 1.1147 - val_accuracy: 0.4250 - val_loss: 1.7241
7/7 ━━━━━━━━━━━━━━━━━━━━ 2s 311ms/step - accuracy: 0.5341 - loss: 1.7465
Test Accuracy: 0.5299999713897705
Saved artifact at '/tmp/tmpje7ope8n'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_10')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


OSError: SavedModel file does not exist at: recyclable_model/{saved_model.pbtxt|saved_model.pb}